In [ ]:
!pip install mlflow boto3 awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2

In [ ]:
from google.colab import userdata

AWS_ACCESS_KEY_ID = userdata.get('AWS_ACCESS_KEY_ID')
AWS_DEFAULT_REGION = userdata.get('AWS_DEFAULT_REGION')
AWS_SECRET_ACCESS_KEY = userdata.get('AWS_SECRET_ACCESS_KEY')

In [ ]:
!aws configure set AWS_ACCESS_KEY_ID $AWS_ACCESS_KEY_ID
!aws configure set AWS_SECRET_ACCESS_KEY $AWS_SECRET_ACCESS_KEY
!aws configure set AWS_DEFAULT_REGION $AWS_DEFAULT_REGION

In [ ]:
import mlflow

In [ ]:
IP = "3.19.222.199"
PROTOCOL = "http"
PORT = "5000"

In [ ]:
TRACKING_URI = f"{PROTOCOL}://{IP}:{PORT}/"
EXPERIMENT_NAME = "EXP 3 - TF-IDF Trigram max_features"

In [ ]:
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

2025/10/13 20:49:11 INFO mlflow.tracking.fluent: Experiment with name 'EXP 3 - TF-IDF Trigram max_features' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://my-mlflow-bucket-1299/423332370002794235', creation_time=1760388551823, experiment_id='423332370002794235', last_update_time=1760388551823, lifecycle_stage='active', name='EXP 3 - TF-IDF Trigram max_features', tags={}>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
from mlflow.models import infer_signature
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [ ]:
df = pd.read_csv('dataset.csv')
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [ ]:
df = df.dropna()
df = df.drop_duplicates()

In [ ]:
df[df['clean_comment'].str.strip() == ''].sum()

,0
clean_comment,0
category,0


In [ ]:
def run_experiment_tfidf_max_features(max_features):
  ngram_range = (1,3)

  vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

  X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'],
                                                      test_size=0.2,random_state=42)
  X_train = vectorizer.fit_transform(X_train)
  X_test = vectorizer.transform(X_test) #fitting would be `cheating` ie data leakage

  with mlflow.start_run() as run:
    #tags for experiment and run
    mlflow.set_tag("mlflow.runName",f"TF-IDF_Trigrams_max_features_{max_features}")
    mlflow.set_tag("experiment_type", "feature_engineering")
    mlflow.set_tag("model_type","RandomForestClassifier")

    #description
    mlflow.set_tag("Description",f"RandomForest with TF-IDF Trigrams, max_features = {max_features}")

    #log vectorizer parameters
    mlflow.log_param("vectorizer_type","TF-IDF")
    mlflow.log_param("ngram_range", ngram_range)
    mlflow.log_param("vectorizer _max_features", max_features)

    n_estimators = 200
    max_depth = 15

    #log Random Forest Parmeters
    mlflow.log_param("n_estimators",n_estimators)
    mlflow.log_param("max_depth", max_depth)

    #Initialize and train the model
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42,)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    #log accuracy
    accuracy =  accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

    #log classification report
    classification_rep = classification_report(y_test,y_pred, output_dict=True)
    for label, metrics in classification_rep.items():
      if isinstance(metrics, dict):
        for metric, value in metrics.items():
          mlflow.log_metric(f"{label}_{metric}", value)

    #log confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize = (8,6))
    sns.heatmap(conf_matrix, annot = True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix: TF-IDF Trigrams, max_features = {max_features}")
    file_name = "confusion_matrix.png"
    plt.savefig(file_name)
    mlflow.log_artifact(file_name)
    plt.close()

    mlflow.sklearn.log_model(
        model,
        name = f"random_forest_model_tfidf_trigrams_{max_features}",
        signature=infer_signature(X_test[:10], y_test[:10])
    )



In [ ]:
max_features_values = [1_000,2_000,3_000,4_000,5_000,6_000,7_000,8_000,9_000,10_000]

for max_feature in max_features_values:
  run_experiment_tfidf_max_features(max_feature)

/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run TF-IDF_Trigrams_max_features_1000 at: http://3.19.222.199:5000/#/experiments/423332370002794235/runs/f680fff73a2c4cc5af844fc7b6db1c93
🧪 View experiment at: http://3.19.222.199:5000/#/experiments/423332370002794235


/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run TF-IDF_Trigrams_max_features_2000 at: http://3.19.222.199:5000/#/experiments/423332370002794235/runs/a396274b4e0744d3b5647f22aca8ee7a
🧪 View experiment at: http://3.19.222.199:5000/#/experiments/423332370002794235


/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


...

🏃 View run TF-IDF_Trigrams_max_features_8000 at: http://3.19.222.199:5000/#/experiments/423332370002794235/runs/221b8d11329a45f1a798b846ed111f52
🧪 View experiment at: http://3.19.222.199:5000/#/experiments/423332370002794235


/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run TF-IDF_Trigrams_max_features_9000 at: http://3.19.222.199:5000/#/experiments/423332370002794235/runs/02c1426eb7fe4ca68c91be62a8fef51f
🧪 View experiment at: http://3.19.222.199:5000/#/experiments/423332370002794235


/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run TF-IDF_Trigrams_max_features_10000 at: http://3.19.222.199:5000/#/experiments/423332370002794235/runs/433466368579439a8d29f042865dacf7
🧪 View experiment at: http://3.19.222.199:5000/#/experiments/423332370002794235
